In [4]:
import os
import logging
import pandas as pd
from collections import defaultdict

In [5]:
def get_sample_files(path,outfile='samples.tsv'):
    samples = defaultdict(dict)
    seen = set()
    for dir_name, sub_dirs, files in os.walk(os.path.abspath(path)):
        for fname in files:

            if ".fastq" in fname or ".fq" in fname:

                sample_id = fname.split(".fastq")[0].split(".fq")[0]

                sample_id = sample_id.replace("_R1", "").replace("_r1", "").replace("_R2", "").replace("_r2", "")
                sample_id = sample_id.replace("_", "-").replace(" ", "-")

                fq_path = os.path.join(dir_name, fname)

                if fq_path in seen: continue

                if "_R2" in fname or "_r2" in fname:

                    if 'R2' in samples[sample_id]:
                        logging.error(f"Duplicate sample {sample_id} was found after renaming; skipping... \n Samples: \n{samples}")

                    samples[sample_id]['R2'] = fq_path
                else:
                    if 'R1' in samples[sample_id]:
                        logging.error(f"Duplicate sample {sample_id} was found after renaming; skipping... \n Samples: \n{samples}")

                    samples[sample_id]['R1'] = fq_path


    samples= pd.DataFrame(samples).T

    if samples.isna().any().any():
        logging.error(f"Missing files:\n {samples}")

    if os.path.exists(outfile):
        logging.error(f"Output file {outfile} already exists I don't date to overwrite it.")
        exit(1)
    else:
        samples.to_csv(outfile,sep='\t')


    return samples

In [6]:
get_sample_files("/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/00_RAW_analysis")

,R2,R1
AUPO-187,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...
AUPO-178,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...
AUPO-186,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...
AUPO-179,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...
AUPO-182,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...
AUPO-180,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...
AUPO-184,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...
AUPO-183,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...
AUPO-181,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...
AUPO-185,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...,/WORKING_DIRECTORY/lvelo/microbiology/LUMI_2/0...


In [5]:
! pwd

/WORKING_DIRECTORY/lvelo/microbiology/LUMI
